In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.preprocessing import *
from sklearn.metrics import *
import torch
from torch import nn
from torch.utils.data import *
from torchvision import datasets
from torchvision.transforms import *


In [ ]:


root_results_dir = "/Users/newuser/Projects/robust-algo-trader/data/trades_seq_EURUSD_H1_2007_2023.csv"
df = pd.read_csv(f"{root_results_dir}")

# take first 4096 rows
df = df.iloc[:6144]

y = df["label"]
X = df[["position", "RSI",  "ADX","ATR", "MFI", "CCI", "AROON_Oscillator"]]


In [ ]:

df

In [ ]:

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
split_index = int(len(X) * 0.6)  # 60% training, 40% testing
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# convert to tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.tensor(y_train.values.astype(np.float32))
y_test = torch.tensor(y_test.values.astype(np.float32))

batch_size = 64

# Create DataLoader objects for training and testing
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
X_train.size(), y_train.size(), X_test.size(), y_test.size()

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device = "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7 , 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

    
input_size = len(X.columns)
hidden_size = 64
output_size = 1
model = NeuralNetwork().to(device)


class PrecisionLoss(nn.Module):
    def __init__(self, weight=None, size_average=None, reduce=None, reduction='mean'):
        super(PrecisionLoss, self).__init__()

        # You can customize additional parameters if needed
        self.loss_function = nn.BCELoss(weight, size_average, reduce, reduction)

    def forward(self, input, target, threshold=0.5):
        # Calculate the binary cross-entropy loss
        bce_loss = self.loss_function(input, target)

        # Convert probabilities to binary predictions using a threshold
        binary_predictions = (input > threshold).float()

        # Calculate precision
        true_positives = torch.sum(binary_predictions * target)
        false_positives = torch.sum(binary_predictions * (1 - target))

        # Calculate precision and add it as a part of the loss
        precision = true_positives / (true_positives + false_positives + 1e-10)
        precision_loss = 1 - precision  # Invert precision to minimize the loss

        # Combine BCE loss and precision loss
        total_loss = bce_loss + precision_loss

        return total_loss

# Example usage:
# criterion = PrecisionLoss()
# loss = criterion(output, targ



class CustomBCELoss(nn.Module):
    def __init__(self, weight=None, reduction='mean'):
        super(CustomBCELoss, self).__init__()
        self.weight = weight
        self.reduction = reduction

    def forward(self, input, target):
        # Compute the binary cross entropy loss
        bce_loss = nn.functional.binary_cross_entropy(input, target, reduction='none')
        # Compute the weight factor based on the input and target
        weight_factor = torch.abs(input - target) ** 2
        # Multiply the loss by the weight and the weight factor
        if self.weight is not None:
            weighted_loss = self.weight * weight_factor * bce_loss
        else:
            weighted_loss = weight_factor * bce_loss
        # Apply the reduction method
        if self.reduction == 'mean':
            return torch.mean(weighted_loss)
        elif self.reduction == 'sum':
            return torch.sum(weighted_loss)
        else:
            return weighted_loss


# loss_fn = CustomBCELoss(weight=3)
# loss_fn = PrecisionLoss()
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

In [ ]:
# Define test function
def test(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            label = y.float().view(-1, 1)
            # check the size of the prediction and label
            # print("pred")
            # print(pred.size())
            # print("label")
            # print(label.size())
            
            current_loss = criterion(pred, label).item()
            test_loss += current_loss
            correct += (pred > 0.5).eq(y.view_as(pred)).sum().item()
    test_loss /= num_batches
    # correct /= size
    print(f"Test Loss: {test_loss}")

# Train and test the model
# epochs = 500_000_000
epochs = 500
# epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train()
    loss_sum = 0
    correct = 0
    for batch, (X, y) in enumerate(train_dataloader):
        # Compute prediction error
        X, y = X.to(device), y.to(device)
        pred = model(X)
        # print(pred)
        label = y.float().view(-1, 1)
        loss = loss_fn(pred, label)

        optimizer.zero_grad()
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        loss_sum += loss.item()
        
        # if batch % 100 == 0:
            # loss, current = loss.item(), (batch + 1) * len(X)
      
    print(f"Train Loss: {loss_sum / len(train_dataloader)}")
    
    test(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
from sklearn.metrics import *

with torch.no_grad():
    pred = model(X_test)
    pred = pred.detach().numpy()
    pred = pred.reshape(-1)
    print(pred)
    pred = np.where(pred > 0.9, 1, 0)
    print(y_test)
    print(accuracy_score(y_test, pred))
    print(precision_score(y_test, pred))

In [ ]:

# count where pred is 1
pred = pd.Series(pred)
pred.value_counts()

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

